In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15

In [2]:

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Vine_Review_Analysis").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Electronics_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Electronics_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41409413|R2MTG1GCZLR2DK|B00428R89M|     112201306|yoomall 5M Antenn...|     Electronics|          5|            0|          0|   N|                Y|          Five Stars|       As described.| 2015-08-31|
|         US|   49668221|R2HBOEM8LE9928|B000068O48|     734576678|Hosa GPM-103 3.5m...|     Electronics|          5|    

In [4]:
#here is a DataFrame or table for the vine_table
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2MTG1GCZLR2DK|          5|            0|          0|   N|                Y|
|R2HBOEM8LE9928|          5|            0|          0|   N|                Y|
|R1P4RW1R9FDPEE|          5|            1|          1|   N|                Y|
|R1EBPM82ENI67M|          1|            0|          0|   N|                Y|
|R372S58V6D11AT|          5|            1|          1|   N|                Y|
|R1A4514XOYI1PD|          5|            1|          1|   N|                Y|
|R20D9EHB7N20V6|          5|            0|          0|   N|                Y|
|R1WUTD8MVSROJU|          5|            0|          0|   N|                Y|
|R1QCYLT25812DM|          4|            0|          0|   N|                Y|
| R904DQPBCEM7A|          4|            0|          0|   N|     

In [7]:
#The data is filtered to create a DataFrame or table where there are 20 or more total votes
total_votes_20_df = vine_df.filter('total_votes > 20')
total_votes_20_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1FBO737KD9F2N|          5|           19|         23|   N|                Y|
|R3SJTYZBYBG4EE|          4|           99|         99|   N|                Y|
|R248FG65D76D5Y|          1|           42|         53|   N|                Y|
|R3B6BXFKGW52SG|          1|           32|         32|   N|                Y|
| ROTIV4VYL31R4|          5|           26|         26|   N|                Y|
|R3VQ59LD2LSKY7|          5|           20|         25|   N|                Y|
| RIIGLD8JB7PX8|          1|           32|         35|   N|                Y|
|R3MUBV21QV0IJK|          3|           77|         84|   N|                Y|
|R1V5W0X6BKIJYA|          5|          129|        132|   N|                Y|
|R274AKHXHYGIN6|          5|           87|        100|   N|     

In [9]:
# The data is filtered to create a DataFrame or table where the percentage of helpful_votes is equal to or greater than 50% 
import pyspark.sql.functions as F

helpful_votes_df = total_votes_20_df.withColumn('help_percent', F.round((total_votes_20_df.helpful_votes/total_votes_20_df.total_votes)*100,2))
cleaned_help_percent_df = helpful_votes_df.filter('help_percent >= 50')
cleaned_help_percent_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|help_percent|
+--------------+-----------+-------------+-----------+----+-----------------+------------+
|R1FBO737KD9F2N|          5|           19|         23|   N|                Y|       82.61|
|R3SJTYZBYBG4EE|          4|           99|         99|   N|                Y|       100.0|
|R248FG65D76D5Y|          1|           42|         53|   N|                Y|       79.25|
|R3B6BXFKGW52SG|          1|           32|         32|   N|                Y|       100.0|
| ROTIV4VYL31R4|          5|           26|         26|   N|                Y|       100.0|
|R3VQ59LD2LSKY7|          5|           20|         25|   N|                Y|        80.0|
| RIIGLD8JB7PX8|          1|           32|         35|   N|                Y|       91.43|
|R3MUBV21QV0IJK|          3|           77|         84|   N|                Y|       91.67|

<bound method DataFrame.count of DataFrame[review_id: string, star_rating: int, helpful_votes: int, total_votes: int, vine: string, verified_purchase: string, help_percent: double]>

In [10]:
#The data is filtered to create a DataFrame or table where there is a Vine review 
yes_vine_df = cleaned_help_percent_df.filter("vine =='Y'")
yes_vine_df.show()



+--------------+-----------+-------------+-----------+----+-----------------+------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|help_percent|
+--------------+-----------+-------------+-----------+----+-----------------+------------+
| R82QWN2X2OCHB|          5|          176|        208|   Y|                N|       84.62|
|R1UYHBYE6790BU|          5|           44|         53|   Y|                N|       83.02|
|R2J3YLX1L4EH2B|          5|          299|        321|   Y|                N|       93.15|
|R3QDI539WTXKE2|          5|           26|         32|   Y|                N|       81.25|
| RQTPWY6ND2NRV|          4|           37|         37|   Y|                N|       100.0|
| RQZSTE0KOBY2G|          4|           98|        109|   Y|                N|       89.91|
| RF2RNZEJO0K1N|          4|           23|         26|   Y|                N|       88.46|
| ROB6VOW41E8O5|          4|          155|        172|   Y|                N|       90.12|

In [11]:
# The data is filtered to create a DataFrame or table where there isn’t a Vine review
no_vine_df = cleaned_help_percent_df.filter("vine =='N'")
no_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|help_percent|
+--------------+-----------+-------------+-----------+----+-----------------+------------+
|R1FBO737KD9F2N|          5|           19|         23|   N|                Y|       82.61|
|R3SJTYZBYBG4EE|          4|           99|         99|   N|                Y|       100.0|
|R248FG65D76D5Y|          1|           42|         53|   N|                Y|       79.25|
|R3B6BXFKGW52SG|          1|           32|         32|   N|                Y|       100.0|
| ROTIV4VYL31R4|          5|           26|         26|   N|                Y|       100.0|
|R3VQ59LD2LSKY7|          5|           20|         25|   N|                Y|        80.0|
| RIIGLD8JB7PX8|          1|           32|         35|   N|                Y|       91.43|
|R3MUBV21QV0IJK|          3|           77|         84|   N|                Y|       91.67|

In [20]:
#The total number of reviews, the number of 5-star reviews, and the percentage 5-star reviews are calculated for all Vine and non-Vine reviews

# Total number of reviews
total_reviews = cleaned_help_percent_df.count()
print(f'Total Reviews: {total_reviews}')

# Total number of 5-star reviews
total_5_star = cleaned_help_percent_df.filter('star_rating == 5').count()
print(f'Total 5 Star Reviews: {total_5_star}')

# Total number of 5-star yes vine reviews
total_5_star_yes = yes_vine_df.filter('star_rating == 5').count()
print(f'Total 5 Star Paid Reviews: {total_5_star_yes}')

# Total number of 5-star no vine
total_5_star_no = no_vine_df.filter('star_rating == 5').count()
print(f'Total 5 Star Unpaid Reviews: {total_5_star_no}')

#Total 5 star review percentage
percent_5 = round((total_5_star / total_reviews)*100,2)
print(f'Percent of all 5 star reviews: {percent_5}%')

#Total 5 star yes vine review percentage
percent_5_yes = round((total_5_star_yes / yes_vine_df.count())*100,2)
print(f'Percent of paid 5 star reviews: {percent_5_yes}%')

#Total 5 star no vine review percentage
percent_5_no = round((total_5_star_no / no_vine_df.count())*100,2)
print(f'Percent of unpaid 5 star reviews: {percent_5_no}%')


Total Reviews: 47808
Total 5 Star Reviews: 22195
Total 5 Star Paid Reviews: 440
Total 5 Star Unpaid Reviews: 21755
Percent of all 5 star reviews: 46.43%
Percent of paid 5 star reviews: 42.64%
Percent of unpaid 5 star reviews: 46.51%
